# Import necessary libraries

In [1]:
import pandas as pd
import scipy.stats as stats
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix

import forestplot as fp

### Load Data

In [2]:
#original diffexp data
diffexp_df = pd.read_excel('diffexp_230621_CLEMENCE_ANALYSIS2B_2023-06-22_03-27-13.xlsx', sheet_name='diffexp')
#original runkey data - which sample belongs to what group
runkey_df = pd.read_excel('diffexp_230621_CLEMENCE_ANALYSIS2B_2023-06-22_03-27-13.xlsx', sheet_name='runkey')
#LipiDisease Associations
lipidisease_tsv = '/Users/brendansu/PycharmProjects/DMEK_v2/lipidisease_output.tsv'
lipidisease_df = pd.read_table(lipidisease_tsv,sep='\t')
#new diffexp data
diffexp_newmodel_df = pd.read_excel('diffexp_newmodel.xlsx')

### Extract sample grouping data

In [3]:
#outcome mapping DMEK:1, Grouping control and glaucoma together: 0
outcome_mapping = {
    "Control": int(0),
    "Glaucoma": int(0),
    "DMEK": int(1)
}

#define function extract grouping from data
def grouper(value):
    value = str(value)
    return value.split(' ')[0].split('_')[0]

#define function that maps categorical outcome to binary
def map_outcome(group):
    if not isinstance(group, str):
        return None
    for key in outcome_mapping.keys():
        if key in group:
            return int(outcome_mapping[key])
    return None

#apply group extraction
runkey_df['id.group'] = runkey_df['ID'].apply(grouper)
#apply binary apping
runkey_df['id.outcome'] = runkey_df['id.group'].apply(map_outcome)
#remove qc and other samples
runkey_df = runkey_df[runkey_df['id.outcome'].notna()]

#name columns by their sample and pivot the dataframe
samples = runkey_df['runId'].tolist()
column_names = [f'log10expr.{i}' for i in samples]
column_names.insert(0,'id.name')
X_df = diffexp_df[column_names]
X_df = X_df.transpose()

#convert dataframe to numpy matrix, this will be our predictors (X)
X = X_df.iloc[1:].to_numpy(dtype=float)

#convert runkey dataframe to numpy matrix, this will be our outcomes/predictions (y)
y = runkey_df['id.outcome'].to_numpy(dtype=float)

### Split Train/Test Data

In [4]:
#normalize X
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

#Train/test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.1, random_state=42)

## Run L1-Regularized Logistic Regression

In [5]:
from sklearn.linear_model import LogisticRegression

# L1-regularized logistic regression
logistic_l1 = LogisticRegression(penalty='l1', C=1.0, solver='liblinear', random_state=42)
logistic_l1.fit(X_train, y_train)
y_pred = logistic_l1.predict(X_test)
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

print("Coefficients:", logistic_l1.coef_)

from sklearn.linear_model import LogisticRegressionCV

# Using K-Fold Cross Validation on Logistic Regression to find the optimal C
logistic_l1_cv = LogisticRegressionCV(penalty='l1', solver='liblinear', cv=5, random_state=42)
logistic_l1_cv.fit(X_train, y_train)

# The best C value
print(f"Optimal C: {logistic_l1_cv.C_}")

# Refit our L1-logistic regression model with the best C value
logistic_l1 = LogisticRegression(penalty='l1', C=logistic_l1_cv.C_[0], solver='liblinear', random_state=42)
logistic_l1.fit(X_train, y_train)

# Check the coefficients and the classification report again
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

print("Coefficients:", logistic_l1.coef_)

for index, coef in enumerate(logistic_l1.coef_[0]):
    print(f"Coefficient for feature {index}: {coef}")

logistic_l1.coef_.transpose()

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00         2
         1.0       1.00      1.00      1.00         1

    accuracy                           1.00         3
   macro avg       1.00      1.00      1.00         3
weighted avg       1.00      1.00      1.00         3

[[2 0]
 [0 1]]
Coefficients: [[-0.52862032  0.          0.         -0.25132972  0.         -0.43128048
   0.01423747  0.          0.          0.         -0.74450375  0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0. 

array([[-0.16208972],
       [ 0.        ],
       [ 0.        ],
       [-0.17167403],
       [-0.06446221],
       [-0.85213737],
       [ 0.0636914 ],
       [-0.96199718],
       [ 0.        ],
       [-0.0147745 ],
       [-0.82692717],
       [ 0.00981669],
       [ 0.        ],
       [ 0.        ],
       [-0.07849168],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [-0.04502178],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [-0.0132405 ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [-0.1058643 ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0

## Bootstrap to obtain Standard Errors of our Logistic Regression Coefficients

In [6]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.utils import resample

n_bootstrap = 1000  # Number of bootstrap samples
coef_var = []  # Store coefficients for each bootstrap sample

for _ in range(n_bootstrap):
    # Bootstrap sample
    X_sample, y_sample = resample(X_train, y_train)

    # Fit the model to the bootstrap sample
    model = LogisticRegression(penalty='l1', C=1.0, solver='liblinear', random_state=42)
    model.fit(X_sample, y_sample)

    # Store the coefficients
    coef_var.append(model.coef_[0])

# Calculate the standard error (SE) of the coefficients across bootstrap samples
coef_var = np.array(coef_var)
standard_errors = coef_var.std(axis=0)

print("Standard Errors of the coefficients:", standard_errors)
standard_errors.transpose()

Standard Errors of the coefficients: [2.69090619e-01 1.85443898e-02 4.18129948e-01 2.76752486e-01
 1.03211105e-01 7.26322854e-01 2.54303803e-01 1.94218837e-02
 7.41069169e-03 6.66018394e-02 4.21759457e-01 5.11655687e-02
 5.21559402e-02 0.00000000e+00 9.20325513e-03 1.29021606e-01
 3.81346437e-03 5.77528646e-02 5.59209114e-02 6.47151867e-05
 0.00000000e+00 0.00000000e+00 0.00000000e+00 4.21294122e-02
 0.00000000e+00 0.00000000e+00 7.96847589e-03 3.11698853e-04
 9.20458249e-03 1.19252940e-02 3.61024557e-05 0.00000000e+00
 0.00000000e+00 2.09646608e-03 3.54765410e-02 3.55188095e-02
 0.00000000e+00 1.08797345e-02 4.49640093e-03 1.20767743e-02
 0.00000000e+00 0.00000000e+00 6.23296843e-03 0.00000000e+00
 1.76686633e-03 0.00000000e+00 3.09199926e-04 2.94742697e-01
 2.26575606e-01 9.31397412e-02 5.31102062e-03 6.32394331e-03
 1.97948954e-02 2.05679165e-02 1.69132637e-02 0.00000000e+00
 0.00000000e+00 0.00000000e+00 7.45257486e-03 1.37593312e-01
 4.14446457e-02 1.20973467e-01 0.00000000e+00 4.

array([2.69090619e-01, 1.85443898e-02, 4.18129948e-01, 2.76752486e-01,
       1.03211105e-01, 7.26322854e-01, 2.54303803e-01, 1.94218837e-02,
       7.41069169e-03, 6.66018394e-02, 4.21759457e-01, 5.11655687e-02,
       5.21559402e-02, 0.00000000e+00, 9.20325513e-03, 1.29021606e-01,
       3.81346437e-03, 5.77528646e-02, 5.59209114e-02, 6.47151867e-05,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 4.21294122e-02,
       0.00000000e+00, 0.00000000e+00, 7.96847589e-03, 3.11698853e-04,
       9.20458249e-03, 1.19252940e-02, 3.61024557e-05, 0.00000000e+00,
       0.00000000e+00, 2.09646608e-03, 3.54765410e-02, 3.55188095e-02,
       0.00000000e+00, 1.08797345e-02, 4.49640093e-03, 1.20767743e-02,
       0.00000000e+00, 0.00000000e+00, 6.23296843e-03, 0.00000000e+00,
       1.76686633e-03, 0.00000000e+00, 3.09199926e-04, 2.94742697e-01,
       2.26575606e-01, 9.31397412e-02, 5.31102062e-03, 6.32394331e-03,
       1.97948954e-02, 2.05679165e-02, 1.69132637e-02, 0.00000000e+00,
      

In [7]:
diffexp_df['lr.DMEK.coef'] = logistic_l1.coef_.transpose()
diffexp_newmodel_df['lr.DMEK.coef'] = logistic_l1.coef_.transpose()
diffexp_newmodel_df['lr.DMEK.coef.se'] = standard_errors.transpose()
lr_df = diffexp_newmodel_df[diffexp_newmodel_df['lr.DMEK.coef']!=0]
lr_df['lr.DMEK.coef.ll'] = lr_df['lr.DMEK.coef'] - 1.96*lr_df['lr.DMEK.coef.se']
lr_df['lr.DMEK.coef.hl'] = lr_df['lr.DMEK.coef'] + 1.96*lr_df['lr.DMEK.coef.se']
lr_df.to_excel('lr_df.xlsx',index=False)
lr_df.to_csv('lr_df.csv',index=False)
lr_df = lr_df.sort_values('lr.DMEK.coef', ascending=True)

lr_notion_df = lr_df[['id.name','id.description','lm1.groupDMEK.pval', 'lm1.groupGlaucoma.pval','lm1.groupDMEK.coef','lm1.groupGlaucoma.coef', 'id.type','lipid.class','id.lipidmaps', 'id.uniprot','id.pubchem','lr.DMEK.coef','lm1.groupDG.pval','lm1.groupDG.coef']].copy()
lr_notion_df.to_excel('lr_notion_df.xlsx',index=False)
lr_notion_df.to_csv('lr_notion_df.csv',index=False)
lr_df[lr_df['id.type']=='lipid']

,id.index,id.level,id.score1,id.score2,id.name,id.name2,id.description,id.formula,id.uniprot,id.type,...,lm1.groupDC.se,lm1.groupDGC.se,lm1.groupDG.sel,lm1.groupDG.seh,lm1.groupDC.sel,lm1.groupDC.seh,lm1.groupDGC.sel,lm1.groupDGC.seh,lr.DMEK.coef.ll,lr.DMEK.coef.hl
403,404,2.0,0.839,0.840,PE-NMe2 34:1,PE-NMe2 34:1;lvl2=1;lvl3=0,PE-NMe2 16:0_18:1,C41H80NO8P,NaN,lipid,...,[0.02647023 0.03835901],[0.02391931 0.04002466],-0.235815,-0.015899,-0.154729,-0.004362,-0.176004,-0.019107,-0.152771,0.039176
234,235,2.0,0.854,0.946,PE 38:5,PE 38:5;lvl2=1;lvl3=0,PE 18:1_20:4,C43H76NO8P,NaN,lipid,...,[0.07348296 0.10648692],[0.06332642 0.10596537],0.169895,0.725505,0.201349,0.618778,0.217008,0.632392,-0.050876,0.388699
262,263,2.0,0.998,0.998,LysoPE 16:0,LysoPE 16:0;lvl2=1;lvl3=0,LysoPE 16:0,C21H44NO7P,NaN,lipid,...,[0.09430215 0.1366568 ],[0.07200346 0.12048484],0.419094,0.933534,0.114298,0.649993,0.260394,0.732695,0.173690,0.472830
223,224,2.0,0.939,0.939,LysoPC 16:0,LysoPC 16:0;lvl2=2;lvl3=0,LysoPC 16:0,C24H50NO7P,NaN,lipid,...,[0.09487968 0.13749372],[0.06758107 0.11308477],0.417529,0.853585,0.159485,0.698460,0.287665,0.730957,-0.245331,0.931805
258,259,2.0,0.998,0.998,LysoPC 18:1,LysoPC 18:1;lvl2=1;lvl3=0,LysoPC 18:1,C26H52NO7P,NaN,lipid,...,[0.08046719 0.11660804],[0.05967871 0.09986158],0.322248,0.779352,0.095157,0.552261,0.216294,0.607751,0.076660,0.643170
94,95,2.0,0.998,0.999,LysoPC 16:1,LysoPC 16:1;lvl2=1;lvl3=0,LysoPC 16:1,C24H48NO7P,NaN,lipid,...,[0.11432516 0.1656729 ],[0.07750135 0.12968457],0.463038,1.077790,0.439527,1.088964,0.512463,1.020826,-0.479495,1.670586
200,201,2.0,0.985,0.992,LysoPC 14:0,LysoPC 14:0;lvl2=1;lvl3=0,LysoPC 14:0,C22H46NO7P,NaN,lipid,...,[0.11091484 0.16073088],[0.08000222 0.13386933],0.521686,1.073371,0.227704,0.857769,0.379438,0.904206,0.283537,1.077990


## Define function for forest plot

In [ ]:
import matplotlib.pyplot as plt
def forest(df,name):
    fp.forestplot(df, ll='lr.DMEK.coef.ll',
                  hl='lr.DMEK.coef.hl',
                  estimate='lr.DMEK.coef',
                  varlabel='id.name',
                  xlabel='Logistic Regression Coefficient',
                  figsize=(10,len(df)/1.8),
                  rightannote=['id.type'],
                  right_annoteheaders=['Analyte Type'],
                  annote=['id.description','est_ci'],
                  annoteheaders=['Analyte Name',"Est. 95% Confidence Int."],
                  color_alt_rows=True,
                  xticks=[-2.5,-2, -1, 0, 1, 2,2.5],
                  table=True,
                 **{
                     "markersize": 35,  # adjust marker size
                     "xlinestyle": (0, (10, 5)),  # long dash for x-reference line
                     "xlinecolor": "#808080",  # gray color for x-reference line
                     "xtick_size": 12,  # adjust x-ticker fontsize
                     'variable_header': 'Analyte ID'
                    }
                  )
    ax = plt.gca()  # Get current axes
    ax.grid(False)  # Disable gridlines
    ax.set_facecolor('#FFFFFF')
    forest = plt.gcf()
    forest.savefig(f'forest_{name}',bbox_inches='tight')

forest(lr_df,'all')
forest(lr_df[(lr_df['id.type']=='lipid')],'lipid')
forest(lr_df[(lr_df['id.type']=='protein')],'protein')
forest(lr_df[(lr_df['id.type']=='compound')|(lr_df['id.type']=='compound2')|(lr_df['id.type']=='electrolyte')],'metabolite')

## Remapping to separate configurations

In [ ]:
runkey_df['id.group'] = runkey_df['ID'].apply(grouper)

outcome_mapping = {
    "Control": int(0),
    "Glaucoma": int(1),
    "DMEK": int(2)
}

runkey_df['id.outcome'] = runkey_df['id.group'].apply(map_outcome)
runkey_df = runkey_df[runkey_df['id.outcome'].notna()]

samples = runkey_df['runId'].tolist()
column_names = [f'log10expr.{i}' for i in samples]
column_names.insert(0,'id.name')
X_df = diffexp_df[column_names]
X_df = X_df.transpose()
X = X_df.iloc[1:].to_numpy(dtype=float)
y = runkey_df['id.outcome'].to_numpy(dtype=float)

y_reshaped = y.reshape(-1, 1)
train = np.append(X, y_reshaped, axis=1)

train_df = pd.DataFrame(train)

## Kruskal-Wallis Test between all three groups

In [ ]:
control_df = train_df[train_df[617] == 0]
glaucoma_df = train_df[train_df[617] == 1]
dmek_df = train_df[train_df[617] == 2]

kwall_df = pd.DataFrame()
hstat = []
pvalues = []
for i in range(0,617):
    kwall = stats.kruskal(control_df[i].tolist(), glaucoma_df[i].tolist(), dmek_df[i].tolist())
    hstat.append(kwall[0])
    pvalues.append(kwall[1])

kwall_df['H Statistic'] = hstat
kwall_df['P-value'] = pvalues
diffexp_newmodel_df['kwall.h'] = kwall_df['H Statistic']
diffexp_newmodel_df['kwall.pval'] = kwall_df['P-value']
diffexp_newmodel_df.to_excel('diffexp_newmodel.xlsx',index=False)
diffexp_newmodel_df